In [55]:
# IMPORTS AND VERSION CHECK
from pathlib import Path
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import sys
import requests

# PyDESeq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

# Config reproducible
np.random.seed(16)

In [56]:
# ENV AND PATHS

# Load environment variables from .env file
repo_root = Path.cwd().parent
load_dotenv(repo_root / ".env")

# Paths
INPUT_DIR = Path(os.getenv("INPUT_DIR"))
OUTPUT_DIR = Path(os.getenv("OUTPUT_DIR"))

# Inputs
RAW_DIR = (repo_root / INPUT_DIR / "raw" / "functional_annotation").resolve()
META_PATH = (repo_root / INPUT_DIR / "metadata" / "metadata.csv").resolve()

# Outputs
analysis_name = "02_deseq_study_KO"
OUT_ROOT = (repo_root / OUTPUT_DIR / analysis_name).resolve()
OUT_CSV = OUT_ROOT / "csv"
OUT_PLOTS = OUT_ROOT / "plots"
for d in [OUT_ROOT, OUT_CSV, OUT_PLOTS]:
    d.mkdir(parents=True, exist_ok=True)

#Check paths
print(f"Repo root: {repo_root}")
print(f"Input dir: {INPUT_DIR}")
print(f"Output dir: {OUTPUT_DIR}")

Repo root: c:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos
Input dir: data
Output dir: results


In [57]:
# FUNCTIONS

def read_emapper(raw_dir, pattern="ERR*.emapper.annotations"):
    """
    Read all *.emapper.annotations files under `raw_dir` matching `pattern`
    and return a single long DataFrame with columns:
        ['sample_id', 'query', 'KEGG_ko'].
    """
    raw_dir = Path(raw_dir)
    files = sorted(raw_dir.glob(pattern))
    if not files:
        raise FileNotFoundError(f"No files matching '{pattern}' found in {raw_dir}")

    all_dfs = []
    for fp in files:
        m = re.search(r"(ERR\d+)", fp.name, flags=re.I)
        sample_id = m.group(1).upper() if m else fp.stem.split(".")[0]

        header = None
        with open(fp, "r", encoding="utf-8", errors="ignore") as f:
            for line in f:
                if line.startswith("#query"):
                    header = line.lstrip("#").strip().split("\t")
                    break
        if header is None:
            print(f"Skipping {fp.name}: no '#query' header found.")
            continue

        df = pd.read_csv(fp, sep="\t", comment="#", names=header, dtype=str, low_memory=False)
        df = df[["query", "KEGG_ko"]].dropna()
        df = df[df["KEGG_ko"] != "-"]
        df["KEGG_ko"] = df["KEGG_ko"].astype(str).str.split(",")
        df = df.explode("KEGG_ko", ignore_index=True)
        df["KEGG_ko"] = df["KEGG_ko"].str.strip()
        df = df[df["KEGG_ko"] != ""]

        df.insert(0, "sample_id", sample_id)
        all_dfs.append(df[["sample_id", "query", "KEGG_ko"]])

    if not all_dfs:
        raise RuntimeError("No valid annotation files could be read.")

    out = pd.concat(all_dfs, ignore_index=True)
    print(f"Loaded {len(files)} files — {out['sample_id'].nunique()} samples, {len(out):,} rows.")
    return out

def summarize_modules(df):
    """
    Return:
      - long_counts: ['sample_id','KEGG_ko','n_proteins'] (#unique queries per module)
      - matrix: samples x modules (counts)
    """
    long_counts = (
        df.groupby(["sample_id", "KEGG_ko"])["query"]
        .nunique()
        .reset_index(name="n_proteins")
    )
    matrix = (
        long_counts
        .pivot(index="sample_id", columns="KEGG_ko", values="n_proteins")
        .fillna(0)
        .astype(int)
    )
    return long_counts, matrix

def matrix_sparsity(matrix):
    """
    Sparsity (percentage of zeros) in a counts matrix.
    """
    total = matrix.size
    zeros = (matrix == 0).sum().sum()
    return round(100 * zeros / total, 2) if total else 0.0

# Read metadata

def load_metadata(meta_path):
    """
    Load metadata.csv and return DataFrame with columns: ['sample_id', 'group'].
    Expects columns 'NCBI_accession' and 'study_condition'.
    """
    meta = pd.read_csv(meta_path)
    if "NCBI_accession" not in meta.columns or "study_condition" not in meta.columns:
        raise ValueError("Metadata must have columns 'NCBI_accession' and 'study_condition'.")
    meta["sample_id"] = (
        meta["NCBI_accession"].astype(str).str.extract(r"(ERR\d+)", expand=False).str.upper()
    )
    meta["group"] = meta["study_condition"].astype(str).str.strip().str.lower()
    meta = meta.dropna(subset=["sample_id", "group"]).drop_duplicates("sample_id")
    return meta[["sample_id", "group"]]

# Results 

def save_csv(df, out_dir, name):
    """
    Save a DataFrame as CSV inside `out_dir`.
    """
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    file_path = out_dir / f"{name}.csv"
    df.to_csv(file_path, index=True)
    print(f"CSV saved in: {file_path}")

def save_plot(out_dir, name, dpi=300):
    """
    Save the current Matplotlib figure in `out_dir` with timestamp.
    """
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    file_path = out_dir / f"{name}.png"
    plt.savefig(file_path, dpi=dpi, bbox_inches="tight")
    plt.close()
    print(f"Figure saved in: {file_path}")

In [58]:
# LOAD ANNOTATIONS AND BUILD ABUNDANCE MATRIX (E-MAPPER -> LONG AND MATRIX)

# Read all emapper annotations
annotations_long = read_emapper(RAW_DIR) 
print(f"Total KEGG_ko annotations: {len(annotations_long):,}")

# Summarize to long_counts and counts matrix (samples x modules)
long_counts, matrix = summarize_modules(annotations_long)
print(f"Matrix shape (samples x KOs): {matrix.shape} | Sparsity: {matrix_sparsity(matrix)}%")

# Save CSV outputs
save_csv(annotations_long, OUT_CSV, "annotations_long")
save_csv(long_counts, OUT_CSV, "counts_long")
save_csv(matrix, OUT_CSV, "counts_matrix")

Loaded 171 files — 171 samples, 16,586,548 rows.
Total KEGG_ko annotations: 16,586,548
Matrix shape (samples x KOs): (171, 14791) | Sparsity: 64.79%
CSV saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\csv\annotations_long.csv
CSV saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\csv\counts_long.csv
CSV saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\csv\counts_matrix.csv


In [59]:
# KEGG MODULE ANNOTATION AND FILTER  (Skipped)

kmap = None  # KEGG module mapping skipped
long_counts_annot = long_counts

save_csv(long_counts_annot, OUT_CSV, "counts_long_annotated")

CSV saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\csv\counts_long_annotated.csv


In [60]:
# METADATA AND MERGE

# Load metadata
meta = load_metadata(META_PATH)
TARGET_GROUPS = ["control", "schizophrenia"]
meta = meta[meta["group"].isin([g.lower() for g in TARGET_GROUPS])].copy()

# Set index to sample_id
meta = meta.set_index("sample_id").sort_index()
matrix = matrix.sort_index()

# Inner join
joined = matrix.join(meta, how="inner")

if joined.empty:
    raise RuntimeError("Inner join produced an empty set: no overlapping samples between counts and metadata.")

# Split back into DESeq2 objects (counts_df and metadata)
metadata = joined[["group"]].copy()
counts_df = joined.drop(columns=["group"]).astype(int)

# Save unfiltered count matrix
save_csv(counts_df, OUT_CSV, "counts_unfiltered")
print(f"Unfiltered counts saved: {counts_df.shape}")

# Filter low-abundance KOs
counts_filtered = counts_df.loc[:, (counts_df.sum(axis=0) >= 30)]

# Save the filtered count matrix
save_csv(counts_filtered, OUT_CSV, "counts_filtered")
save_csv(metadata, OUT_CSV, "metadata_aligned")

# Report summary
print("DESeq2 inputs (KO):")
print("  counts_unfiltered:", counts_df.shape)
print("  counts_filtered:  ", counts_filtered.shape)
print("  metadata:         ", metadata.shape)
print("Groups:", metadata["group"].value_counts().to_dict())

CSV saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\csv\counts_unfiltered.csv
Unfiltered counts saved: (171, 14791)
CSV saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\csv\counts_filtered.csv
CSV saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\csv\metadata_aligned.csv
DESeq2 inputs (KO):
  counts_unfiltered: (171, 14791)
  counts_filtered:   (171, 6315)
  metadata:          (171, 1)
Groups: {'schizophrenia': 90, 'control': 81}


In [61]:
# COUNT MATRIX + METADATA
# Combine original counts_matrix (samples x modules) with metadata and save
counts_matrix_with_meta = metadata.join(matrix, how="inner")
save_csv(counts_matrix_with_meta, OUT_CSV, "counts_matrix_with_metadata")

CSV saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\csv\counts_matrix_with_metadata.csv


In [62]:
# DIFFERENTIAL ANALYSIS (PyDESeq2)

# Configure groups and set reference
metadata = metadata.copy()
metadata["group"] = pd.Categorical(
    metadata["group"].str.lower(),
    categories=["control", "schizophrenia"],
    ordered=True
)

# Initialize DESeqDataSet
dds = DeseqDataSet(
    counts=counts_filtered,
    metadata=metadata,      
    design="~ group",
    refit_cooks=True
)
dds.deseq2()

# Contrast: schizophrenia vs control
ds = DeseqStats(
    dds, 
    contrast=["group", "schizophrenia", "control"],
    alpha=0.05,
    cooks_filter=True,
    independent_filter=True
)
ds.summary()

Fitting size factors...
... done in 0.11 seconds.



Using None as control genes, passed at DeseqDataSet initialization


Fitting dispersions...
... done in 6.74 seconds.

Fitting dispersion trend curve...
... done in 0.43 seconds.

Fitting MAP dispersions...
... done in 8.93 seconds.

Fitting LFCs...
... done in 4.32 seconds.

Calculating cook's distance...
... done in 0.26 seconds.

Replacing 10 outlier genes.

Fitting dispersions...
... done in 0.04 seconds.

Fitting MAP dispersions...
... done in 0.03 seconds.

Fitting LFCs...
... done in 0.03 seconds.

Running Wald tests...


Log2 fold change & Wald test p-value: group schizophrenia vs control
            baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
ko:K00001  22.752686        0.166579  0.057056  2.919599  0.003505  0.216443
ko:K00002   4.358997       -0.030692  0.105641 -0.290533  0.771408       NaN
ko:K00003  50.669251       -0.006142  0.033313 -0.184369  0.853724  0.969300
ko:K00004   3.559560        0.365996  0.130096  2.813266  0.004904       NaN
ko:K00005  25.324443        0.028239  0.057203  0.493662  0.621545  0.915165
...              ...             ...       ...       ...       ...       ...
ko:K22507   0.537556       -0.389476  0.333112 -1.169205  0.242321       NaN
ko:K22508   0.510049       -0.483618  0.334314 -1.446598  0.148010       NaN
ko:K22509   0.496987       -0.146002  0.342536 -0.426239  0.669934       NaN
ko:K22510   0.787701       -0.323969  0.317345 -1.020872  0.307315       NaN
ko:K22511   0.242977       -0.281030  0.593088 -0.473843  0.635612       NaN

[6315 

... done in 2.33 seconds.



In [63]:
# GET RESULTS AND SIGNIFICANT KOs
res_df = ds.results_df.copy()
res_df = res_df.reset_index()
if "KEGG_ko" not in res_df.columns:
    res_df = res_df.rename(columns={res_df.columns[0]: "KEGG_ko"})

res_clean = res_df.dropna(subset=["log2FoldChange", "padj"]).copy()

# Significant list (padj < 0.05)
sig = res_clean[res_clean["padj"] < 0.05].sort_values("padj", na_position="last").copy()

n_total = res_clean.shape[0]
n_sig   = sig.shape[0]
n_up    = int((sig["log2FoldChange"] > 0).sum())
n_down  = int((sig["log2FoldChange"] < 0).sum())

print(f"Total tested KOs: {n_total}")
print(f"Significant (padj < 0.05): {n_sig} | Up: {n_up} | Down: {n_down}")

# Save significant table
case_group = "schizophrenia"
ref_group  = "control"
save_csv(sig, OUT_CSV, f"significant_KOs__{case_group}_vs_{ref_group}")

# Add direction column (up/down regulation)
sig["direction"] = np.where(
    sig["log2FoldChange"] > 0, "up_in_schizophrenia", "down_in_schizophrenia"
)

# Save annotated table
save_csv(sig, OUT_CSV, f"significant_Kos_annotated__{case_group}_vs_{ref_group}")

Total tested KOs: 2397
Significant (padj < 0.05): 9 | Up: 6 | Down: 3
CSV saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\csv\significant_KOs__schizophrenia_vs_control.csv
CSV saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\csv\significant_Kos_annotated__schizophrenia_vs_control.csv


In [64]:
# LFC distribution: seaborn histogram with KDE and mean ± SD lines
lfc = res_clean["log2FoldChange"].astype(float)
mu  = float(lfc.mean())
sd  = float(lfc.std(ddof=1))

plt.figure()
sns.histplot(lfc, bins=50, kde=True)
plt.axvline(mu,             ls="-",  lw=2,   color="black", label=f"Mean = {mu:.2f}")
plt.axvline(mu - sd,        ls="--", lw=1.5, color="gray",  label=f"Mean ± SD ({sd:.2f})")
plt.axvline(mu + sd,        ls="--", lw=1.5, color="gray")
plt.xlabel("log2 Fold Change")
plt.title(f"LFC distribution ({case_group} vs {ref_group}) with Mean ± SD")
plt.legend()
save_plot(OUT_PLOTS, f"hist_log2fc_mean_pm_sd__{case_group}_vs_{ref_group}")

Figure saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\plots\hist_log2fc_mean_pm_sd__schizophrenia_vs_control.png


In [65]:
# VOLCANO AND BARPLOT

sns.set_theme(style="whitegrid", context="notebook", font_scale=1.1)

# Prepare results
res_df = ds.results_df.copy().reset_index()
if "KEGG_Module" not in res_df.columns:
    res_df = res_df.rename(columns={res_df.columns[0]: "KEGG_Module"})

res_clean = res_df.dropna(subset=["log2FoldChange", "padj"]).copy()
res_clean["neglog10_padj"] = -np.log10(res_clean["padj"].clip(lower=1e-300))
res_clean["abs_log2FC"] = res_clean["log2FoldChange"].abs()
res_clean["significance_label"] = np.where(
    res_clean["padj"] < 0.05, "Significant (padj < 0.05)", "Not significant"
)

# Merge KEGG annotation if available
if "kmap" in globals() and isinstance(kmap, pd.DataFrame) and not kmap.empty:
    res_clean = res_clean.merge(kmap, on="KEGG_Module", how="left")
else:
    res_clean["Module_name"] = res_clean["KEGG_Module"]
    res_clean["Module_description"] = "NA"

# Direction
res_clean["direction"] = np.where(
    res_clean["log2FoldChange"] > 0, "up_in_schizophrenia", "down_in_schizophrenia"
)

# 1) Volcano plot

volcano_palette = {
    "Not significant": "#B8B8B8",              
    "Significant (padj < 0.05)": "#7F3C8D",    
}

plt.figure(figsize=(9, 6))
ax = sns.scatterplot(
    data=res_clean,
    x="log2FoldChange",
    y="neglog10_padj",
    hue="significance_label",
    style="significance_label",
    s=35, alpha=0.85,
    palette=volcano_palette,
    edgecolor=None
)
plt.axvline(0, ls="--", lw=1, color="gray")
plt.axhline(-np.log10(0.05), ls="--", lw=1, color="gray")
plt.xlabel("log2 Fold Change (schizophrenia vs control)", fontsize=12)
plt.ylabel("-log10(padj)", fontsize=12)
plt.title("Volcano plot of differential KEGG modules", fontsize=14, pad=15)
plt.legend(
    title="Significance",
    bbox_to_anchor=(1.02, 1),
    loc="upper left",
    borderaxespad=0,
    frameon=False,
    fontsize=10,
    title_fontsize=11
)
plt.tight_layout(rect=[0, 0, 0.85, 1])
save_plot(OUT_PLOTS, "volcano__schizophrenia_vs_control")

# 2) Barplot

topN = 20
if (res_clean["significance_label"] == "Significant (padj < 0.05)").any():
    top = (res_clean[res_clean["significance_label"] == "Significant (padj < 0.05)"]
           .sort_values(["abs_log2FC", "padj"], ascending=[False, True])
           .head(topN)
           .copy())
else:
    top = (res_clean
           .sort_values(["abs_log2FC", "padj"], ascending=[False, True])
           .head(topN)
           .copy())

top["label"] = top["Module_name"].fillna(top["KEGG_Module"])
top = top.sort_values("log2FoldChange", ascending=True)

dir_palette = {
    "down_in_schizophrenia":  "#B8B8B8",  
    "up_in_schizophrenia":   "#7F3C8D",  
}

plt.figure(figsize=(10, max(5, 0.45 * len(top))))
ax = sns.barplot(
    data=top,
    x="log2FoldChange",
    y="label",
    hue="direction",
    dodge=False,
    palette=dir_palette
)
plt.axvline(0, ls="--", lw=1, color="gray")
plt.xlabel("log2 Fold Change (schizophrenia vs control)", fontsize=12)
plt.ylabel("KEGG module", fontsize=12)
plt.title(f"Top {len(top)} KEGG modules by |log2FC|", fontsize=14, pad=15)
plt.legend(
    title="Direction",
    bbox_to_anchor=(1.02, 1),
    loc="upper left",
    borderaxespad=0,
    frameon=False,
    fontsize=10,
    title_fontsize=11
)
plt.tight_layout(rect=[0, 0, 0.83, 1])
save_plot(OUT_PLOTS, "bar_top_modules_by_abs_log2fc__schizophrenia_vs_control")


Figure saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\plots\volcano__schizophrenia_vs_control.png
Figure saved in: C:\Users\Andre\OneDrive\Escritorio\tfm_andrea_ramos\results\02_deseq_study_KO\plots\bar_top_modules_by_abs_log2fc__schizophrenia_vs_control.png
